## Get all Symbol Exchange

In [1]:
import ccxt
import pandas as pd


def get_exchange_symbols(exchange_id):
    """
    Fetch all trading symbols for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of trading pair symbols (e.g., ["BTC/USDT", "ETH/USDT"]).
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        
        # Load markets
        markets = exchange.load_markets()
        
        # Extract symbols
        symbols = list(markets.keys())
        return symbols

    except Exception as e:
        print(f"Error fetching symbols for exchange {exchange_id}: {e}")
        return []

In [3]:

exchange_id = "binance"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

exchange_id = "bitget"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

Total symbols on binance: 3336
Sample symbols: ['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH', 'EOS/ETH', 'SNT/ETH', 'BNT/ETH', 'BCC/BTC', 'GAS/BTC']
Total symbols on bitget: 1276
Sample symbols: ['FTM/EUR', 'CATCH/USDT', 'LUMIA/USDT', 'GOAT/USDT', 'ENA/EUR', 'TON/EUR', 'SXRP/SUSDT', 'SHIB/EUR', 'WLD/EUR', 'DBR/USDT']


In [4]:
def get_top_usdt_symbol_by_volume(exchange_id, top_n=100):
    """
    Fetch the top N cryptocurrencies traded against USDT based on 24h volume.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").
        top_n (int): The number of top cryptocurrencies to fetch (default is 100).

    Returns:
        pd.DataFrame: A DataFrame containing the top N USDT pairs by 24h volume.
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        
        # Fetch all tickers
        tickers = exchange.fetch_tickers()
        
        # Extract relevant data for USDT pairs
        data = []
        for symbol, ticker in tickers.items():
            if symbol.endswith("/USDT"):  # Filter for USDT pairs
                data.append({
                    "symbol": symbol,
                    "volume_24h": ticker.get("quoteVolume", 0),  # 24h trading volume in USDT
                    "price": ticker.get("last", 0),  # Last traded price
                })
        
        # Convert to DataFrame and sort by volume
        df = pd.DataFrame(data)
        df = df.sort_values(by="volume_24h", ascending=False).head(top_n)
        
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [50]:
exchange_id = "binance"  # Replace with your desired exchange ID
top_symbols = get_top_usdt_symbol_by_volume(exchange_id, top_n=100)
binance_symbols = pd.DataFrame(top_symbols)
binance_symbols = binance_symbols.rename(columns={"volume_24h": f"volume_24h_{exchange_id}"})
binance_symbols = binance_symbols.rename(columns={"price": f"price_{exchange_id}"})
binance_symbols = binance_symbols.reset_index(drop=True)
binance_symbols.to_csv(f"/home/ubuntu/project/finance/cex-market-analysis/symbols/{exchange_id}.csv")


exchange_id = "bitget"  # Replace with your desired exchange ID
top_symbols = get_top_usdt_symbol_by_volume(exchange_id, top_n=100)
bitget_symbols = pd.DataFrame(top_symbols)
bitget_symbols = bitget_symbols.rename(columns={"volume_24h": f"volume_24h_{exchange_id}"})
bitget_symbols = bitget_symbols.rename(columns={"price": f"price_{exchange_id}"})
bitget_symbols = bitget_symbols.reset_index(drop=True)
bitget_symbols.to_csv(f"/home/ubuntu/project/finance/cex-market-analysis/symbols/{exchange_id}.csv")
df = pd.merge(binance_symbols, bitget_symbols, on=['symbol'])
df

,symbol,volume_24h_binance,price_binance,volume_24h_bitget,price_bitget
0,BTC/USDT,1.090674e+09,93938.640000,4.331536e+08,93928.950000
1,USDC/USDT,7.653338e+08,1.002100,1.883515e+07,1.002200
2,ETH/USDT,6.877967e+08,3331.560000,2.903882e+08,3332.090000
3,XRP/USDT,5.064697e+08,2.228400,1.129885e+08,2.227500
4,PEPE/USDT,4.257723e+08,0.000019,2.519323e+07,0.000019
5,SOL/USDT,3.561597e+08,189.760000,8.480465e+07,189.750000
6,DOGE/USDT,2.223381e+08,0.318080,5.557130e+07,0.318130
7,XLM/USDT,1.682866e+08,0.393900,3.483910e+07,0.393700
8,HBAR/USDT,1.295807e+08,0.285020,5.437685e+06,0.285500
9,SUI/USDT,1.123385e+08,4.166000,1.355544e+07,4.165600


In [51]:
df.to_csv(f"/home/ubuntu/project/finance/cex-market-analysis/symbols/symbols.csv")


In [41]:
df = pd.read_csv("/home/ubuntu/project/finance/cex-market-analysis/symbols/bitget.csv")

symbols = df["symbol"]

In [45]:
def get_exchange_timeframes(exchange_id):
    """
    Get the list of available timeframes for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of available timeframes for the exchange.
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})

        # Check if the exchange supports timeframes
        if hasattr(exchange, 'timeframes') and exchange.timeframes:
            return list(exchange.timeframes.keys())
        else:
            return []
    except Exception as e:
        print(f"Error fetching timeframes for {exchange_id}: {e}")
        return []

# Example Usage
exchange_id = "bitget"  # Replace with your desired exchange ID
timeframes = get_exchange_timeframes(exchange_id)
print(f"Available timeframes for {exchange_id}: {timeframes}")

Available timeframes for bitget: ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '12h', '1d', '3d', '1w', '1M']
